In [60]:
import aocd
import dataclasses
import numpy as np
import enum

real_data = aocd.get_data(day=10, year=2022)
test_data = """addx 15
addx -11
addx 6
addx -3
addx 5
addx -1
addx -8
addx 13
addx 4
noop
addx -1
addx 5
addx -1
addx 5
addx -1
addx 5
addx -1
addx 5
addx -1
addx -35
addx 1
addx 24
addx -19
addx 1
addx 16
addx -11
noop
noop
addx 21
addx -15
noop
noop
addx -3
addx 9
addx 1
addx -3
addx 8
addx 1
addx 5
noop
noop
noop
noop
noop
addx -36
noop
addx 1
addx 7
noop
noop
noop
addx 2
addx 6
noop
noop
noop
noop
noop
addx 1
noop
noop
addx 7
addx 1
noop
addx -13
addx 13
addx 7
noop
addx 1
addx -33
noop
noop
noop
addx 2
noop
noop
noop
addx 8
noop
addx -1
addx 2
addx 1
noop
addx 17
addx -9
addx 1
addx 1
addx -3
addx 11
noop
noop
addx 1
noop
addx 1
noop
noop
addx -13
addx -19
addx 1
addx 3
addx 26
addx -30
addx 12
addx -1
addx 3
addx 1
noop
noop
noop
addx -9
addx 18
addx 1
addx 2
noop
noop
addx 9
noop
noop
noop
addx -1
addx 2
addx -37
addx 1
addx 3
noop
addx 15
addx -21
addx 22
addx -6
addx 1
noop
addx 2
addx 1
noop
addx -10
noop
noop
addx 20
addx 1
addx 2
addx 2
addx -6
addx -11
noop
noop
noop"""

In [180]:
from typing import Sequence, Union
import itertools

@dataclasses.dataclass
class Register:
    """A state that store 1 `int`.
    
    Args:
        value: the initial value of the state.
        instructions: the list of instructions in `str`.
    """
    value: int
    instructions: Sequence[str]
        
    def __post_init__(self) -> None:
        self.sum_strength = 0
        self.queue = []
        self.signal_strength_schedule = -20
        self.clock = 1

    def addx(self, v: int) -> None:
        """Execute the `addx` instruction. Costs 2 cycles.
        
        Args:
            v: the value to add to the .
        """
        self.queue.extend([0, v])
        
    def noop(self) -> None:
        """Does nothing, costs 1 cycle."""
        self.queue.append(0)
        
    def tik(self) -> None:
        """Adds 1 cycle to clock. Executes the command in queue."""

        self.signal_strength_schedule += 1
        if self.signal_strength_schedule % 40 == 0:
            self.sum_strength += self.get_signal()
        
        self.clock += 1
        self.value += self.queue.pop(0)


    def get_signal(self) -> int:
        """Calculates and returns the signal strength.
        
        Returns:
            signal strength
        """
        return self.value * self.clock

    def start_program(self) -> None:
        """Starts the program."""
        # load all instructions
        for instruction in self.instructions:
            if "noop" in instruction:
                self.noop()
            elif "addx" in instruction:
                self.addx(int(instruction.split(" ")[-1]))
                
        while(len(self.queue) > 0):
            self.tik()
    
@dataclasses.dataclass
class SolverA:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

    def __post_init__(self):
        self.lines = self.raw_data.split("\n")
        
    def find_answer(self) -> int:
        """Finds the answer.
        
        Returns:
            The answer.
        """
        reg = Register(1, self.lines)
        reg.start_program()
        return reg.sum_strength

In [181]:
SolverA(test_data).find_answer()

13140

In [104]:
answer = SolverA(real_data).find_answer()
aocd.submit(answer, part="a", day=10, year=2022)

Part a already solved with same answer: 15220


In [172]:
from typing import Sequence, Union
import itertools

@dataclasses.dataclass
class Register:
    """A state that store 1 `int`.
    
    Args:
        value: the initial value of the state.
        instructions: the list of instructions in `str`.
    """
    value: int
    instructions: Sequence[str]
        
    def __post_init__(self) -> None:
        self.queue = []
        self.drawing = ""
        self.clock = 0

    def addx(self, v: int) -> None:
        """Execute the `addx` instruction. Costs 2 cycles.
        
        Args:
            v: the value to add to the .
        """
        self.queue.extend([0, v])
        
    def noop(self) -> None:
        """Does nothing, costs 1 cycle."""
        self.queue.append(0)
        
    def tik(self) -> None:
        """Adds 1 cycle to clock. Executes the command in queue."""
        self.draw()
        self.clock += 1
        self.value += self.queue.pop(0)
        
    def draw(self) -> None:
        """Draws the pixel."""
        on = bool(self.get_sprite()[self.clock % 40])
        self.drawing += "#" if on else "."
        if (self.clock + 1) % 40 == 0:
            self.drawing = self.drawing + "\n"

    def start_program(self) -> None:
        """Starts the program."""
        # load all instructions
        for instruction in self.instructions:
            if "noop" in instruction:
                self.noop()
            elif "addx" in instruction:
                self.addx(int(instruction.split(" ")[-1]))
                
        while(len(self.queue) > 0):
            self.tik()
            
    def get_sprite(self):
        """Outputs the sprit image (1d array)."""
        disp = np.zeros(40).astype(int)
        disp[self.value - 1 : self.value + 2] = 1
        return disp
        
@dataclasses.dataclass
class SolverB:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

    def __post_init__(self):
        self.lines = self.raw_data.split("\n")
        
    def find_answer(self) -> None:
        """Finds the answer.
        
        Prints the CRT
        """
        reg = Register(1, self.lines)
        reg.start_program()
        print(reg.drawing)
        return None

In [173]:
SolverB(test_data).find_answer()

##..##..##..##..##..##..##..##..##..##..
###...###...###...###...###...###...###.
####....####....####....####....####....
#####.....#####.....#####.....#####.....
######......######......######......####
#######.......#######.......#######.....



In [174]:
answer = SolverB(real_data).find_answer()

###..####.####.####.#..#.###..####..##..
...#.#.......#.#....#.#..#..#.#....#..#.
...#.###....#..###..##...###..###..#..#.
###..#.....#...#....#.#..#..#.#....####.
..#..#....#....#....#.#..#..#.#....#..#.
...#.#....####.####.#..#.###..#....#..#.



In [175]:
aocd.submit("RFZEKBFA", part="b", day=10, year=2022)

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 10! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>